In [ ]:
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess
import os
from mothur_py import Mothur
from shutil import copy
import warnings
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from ipysankeywidget import SankeyWidget
warnings.filterwarnings("ignore")

In [ ]:
def get_fasta(fastq):
    SeqIO.convert(fastq, "fastq", fastq[:-6]+'.fasta', "fasta")
    return fastq[:-6]+'.fasta'

In [ ]:
# get_fasta('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.fastq')

In [ ]:
def get_bedoutput_ITS(reference_fasta, primer_fasta):
    """The purpose of this function is to generate a fasta file of the reference genomeregions between primers, 
    which should correspond with the region sequenced using rhe MinION. This requires first the generation of a 
    database using the nucleotide makeblastdb command, followed by running a blast search across the database 
    using the primer sequence. These command line commands are processed through subprocess. Subsequently, the 
    resulting outfile from the blast is read in as a dataframe and this dataframe is manipulated in order to 
    utilise only the minimum evalues. Each forward sequence is then compared to each reverse sequence, given they
    are on the same contig and the resultant read would be within 2000-4000 bp long. These results are saved as 
    a bedfile for use in the get_bedfasta function"""
    cmd = 'makeblastdb -in %s -dbtype nucl -out %s' % (reference_fasta, reference_fasta[:-6]+'_db')
    subprocess.getoutput(cmd)
    cmd2 = 'blastn -query %s -db %s -evalue=100000 -task "blastn-short" -outfmt 6 > %s' % (primer_fasta, reference_fasta[:-6]+'_db', reference_fasta[:-6]+'.outfmt6')
    subprocess.getoutput(cmd2)
    df = pd.read_csv(reference_fasta[:-6]+'.outfmt6', sep="\t", header=None, names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])
    forward_df = df.loc[(df['qseqid']=='forward')]
    forward_df = forward_df.loc[df['evalue']==min(forward_df['evalue'])]
    reverse_df = df.loc[(df['qseqid']=='reverse')]
    reverse_df = reverse_df.loc[df['evalue']==min(reverse_df['evalue'])]
    forward_bed = pd.DataFrame(columns=['chrom', 'chromStart', 'chromEnd'])
    for i in range(0, len(forward_df)):
        if forward_df['sstart'][i] <= forward_df['send'][i]:
            forward_bed = forward_bed.append({'chrom': forward_df['sseqid'][i],'chromStart': (forward_df['sstart'][i])-1, 'chromEnd': forward_df['send'][i]}, ignore_index=True)
        else:
            forward_bed = forward_bed.append({'chrom': forward_df['sseqid'][i],'chromStart': (forward_df['send'][i])-1, 'chromEnd': forward_df['sstart'][i]}, ignore_index=True)
    forward_bed.sort_values(['chromStart'])
    reverse_bed = pd.DataFrame(columns=['chrom', 'chromStart', 'chromEnd'])
    for i in reverse_df.index:
        if reverse_df['sstart'][i] < reverse_df['send'][i]:
            reverse_bed = reverse_bed.append({'chrom': reverse_df['sseqid'][i],'chromStart': (reverse_df['sstart'][i])-1, 'chromEnd': reverse_df['send'][i]}, ignore_index=True)
        else:
            reverse_bed = reverse_bed.append({'chrom': reverse_df['sseqid'][i],'chromStart': (reverse_df['send'][i])-1, 'chromEnd': reverse_df['sstart'][i]}, ignore_index=True)
    reverse_bed.sort_values(['chromStart'])
    intervals_list = []
    for row in forward_bed.itertuples(index=True, name='Pandas'):
        counter = 0
        for rows in reverse_bed.itertuples(index=True, name='Pandas'):
            # Here lies the difference between the ITS and TEF versions of this function - for ITS the read length range is 2000-4000
            if getattr(row, 'chrom') == getattr(rows, 'chrom') and 2000 < np.absolute(getattr(row, 'chromEnd') - getattr(rows, 'chromStart')) < 4000:
                counter += 1
                intervals_list.append([forward_bed['chrom'][getattr(row, 'Index')], forward_bed['chromEnd'][getattr(row, 'Index')], reverse_bed['chromStart'][getattr(rows, 'Index')]])
        if counter > 1:
            print("WARNING")
    intervals_frame = pd.DataFrame(data=intervals_list, columns=['chrom', 'chromStart', 'chromEnd'])
    print(intervals_frame)
    intervals_frame.to_csv(reference_fasta[:-6]+'.ITS.bedfile', sep='\t', header=False, index=False)
#     lines = []
#     for i in range(0, len(intervals_frame)):
#         lines.append(['%i\t%i\t%i' % (intervals_frame['chrom'][i], intervals_frame['chromStart'][i], intervals_frame['chromEnd'][i])]) 
#     with open(reference_fasta[:-6]+'.ITS.bedfile', 'w') as f:
#         for i in range(0, len(lines)):
#             f.writelines(lines[i])
#             f.writelines('\n')
    return reference_fasta[:-6]+'.ITS.bedfile'

In [ ]:
# get_bedoutput_ITS('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.reference.fasta', '/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/ITS_primers.fasta')

In [ ]:
def get_bedfasta(bedfile, reference_fasta):
    cmd2 = 'bedtools getfasta -fo %s -fi %s -bed %s' % (reference_fasta[:-6]+'.ITS.bedoutput.fasta', reference_fasta, bedfile)
    subprocess.getoutput(cmd2)
    return reference_fasta[:-6]+'.ITS.bedoutput.fasta'

In [ ]:
# get_bedfasta('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.reference.ITS.bedfile', '/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.reference.fasta')

In [ ]:
def length_filter(fasta, minimum, maximum):
    fasta_dict = SeqIO.to_dict(SeqIO.parse(fasta, "fasta"))
    short_dict = fasta_dict.copy()
    for key in fasta_dict:
        if len(fasta_dict[key].seq) not in range(minimum, maximum + 1):
            del short_dict[key]
    SeqIO.write(short_dict.values(), fasta[:-6]+'.filtered_base_%s-%s.fasta' % (minimum, maximum), "fasta")
    return fasta[:-6]+'.filtered_base_%s-%s.fasta' % (minimum, maximum)

In [ ]:
# length_filter('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.fasta', 2000, 4000)

In [ ]:
def pcr_seqs(filtered_fasta, oligos):
    tmp = SeqIO.to_dict(SeqIO.parse(filtered_fasta, "fasta"))
    new_name = filtered_fasta+'.mothur5.%s.fasta' % (oligos[-10:-7])
    SeqIO.write(tmp.values(), new_name, "fasta")
    m = Mothur()
    m.pcr.seqs(fasta=new_name, oligos=oligos, pdiffs=5, rdiffs=5)

In [ ]:
# pcr_seqs('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.filtered_base_2000-4000.fasta', '/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/ITS.oligos')

In [ ]:
def pairwise_aln(reference_bed_output, mothur_query_fasta):
    gen_dict = SeqIO.to_dict(SeqIO.parse(reference_bed_output, "fasta"))
    print(gen_dict)
    gen_dict_seqs = []
    for key in gen_dict:
        gen_dict_seqs.append(gen_dict[key].seq)
    print(gen_dict_seqs)
    q_dict = SeqIO.to_dict(SeqIO.parse(mothur_query_fasta, "fasta"))
    error5w = {}
    error5w_key_entries = ['keys', 'alnslengths', 'aap', 'acp', 'agp', 'atp', 'agapp', 'cap', 'ccp', 'cgp', 'ctp', 'cgapp', 'gap', 'gcp', 'ggp', 'gtp', 'ggapp', 'tap', 'tcp', 'tgp', 'ttp', 'tgapp', 'gapap', 'gapcp', 'gapgp', 'gaptp', 'expeca', 'expecc', 'expecg', 'expect']
    for item in error5w_key_entries:
        error5w[item] = []
    for key in q_dict:
        error5w['keys'].append(key)
        alignments = pairwise2.align.globalms(gen_dict_seqs[0], q_dict[key].seq, 1, -1, -1, 0)
        total = 0
        aa = 0
        ac = 0
        ag = 0
        at = 0
        agap = 0
        ca =0
        cc=0
        cg=0
        ct=0
        cgap=0
        ga=0
        gc=0
        gg=0
        gt=0
        ggap=0
        ta=0
        tc=0
        tg=0
        tt=0
        tgap=0
        gapa=0
        gapc=0
        gapg=0
        gapt=0
        tmpa=0
        tmpc=0
        tmpg=0
        tmpt=0
        comparison = alignments[0][0:2]
        for i in range(0, len(comparison[0])):
            if comparison[0][i] == "A":
                tmpa += 1
                if comparison[1][i] == "C":
                    ac+=1
                    total += 1
                elif comparison[1][i] == "G":
                    ag+=1
                    total += 1
                elif comparison[1][i] == "T":
                    at+=1
                    total += 1
                elif comparison[1][i] == "A":
                    aa+=1
                    total += 1
                else:
                    agap+=1
                    total += 1
            if comparison[0][i] == "C":
                tmpc+=1
                if comparison[1][i] == "A":
                    ca+=1
                    total += 1
                elif comparison[1][i] == "G":
                    cg+=1
                    total += 1
                elif comparison[1][i] == "T":
                    ct+=1
                    total += 1
                elif comparison[1][i] == "C":
                    cc+=1
                    total += 1
                else:
                    cgap+=1
                    total += 1
            if comparison[0][i] == "G":
                tmpg+=1
                if comparison[1][i] == "A":
                    ga+=1
                    total += 1
                elif comparison[1][i] == "C":
                    gc+=1
                    total += 1
                elif comparison[1][i] == "T":
                    gt+=1
                    total += 1
                elif comparison[1][i] == "G":
                    gg+=1
                    total += 1
                else:
                    ggap+=1
                    total += 1
            if comparison[0][i] == "T":
                tmpt+=1
                if comparison[1][i] == "A":
                    ta+=1
                    total += 1
                elif comparison[1][i] == "C":
                    tc+=1
                    total += 1
                elif comparison[1][i] == "G":
                    tg+=1
                    total += 1
                elif comparison[1][i] == "T":
                    tt+=1
                    total += 1
                else:
                    tgap+=1
                    total += 1
            if comparison[0][i] == "-":
                if comparison[1][i] == "A":
                    gapa+=1
                    total += 1
                elif comparison[1][i] == "C":
                    gapc+=1
                    total += 1
                elif comparison[1][i] == "G":
                    gapg+=1
                    total += 1
                elif comparison[1][i] == "T":
                    gapt+=1
                    total += 1
        error5w['acp'].append(100*ac/total)
        error5w['agp'].append(100*ag/total)
        error5w['atp'].append(100*at/total)
        error5w['agapp'].append(100*agap/total)
        error5w['aap'].append(100*aa/total)
        error5w['cap'].append(100*ca/total)
        error5w['cgp'].append(100*cg/total)
        error5w['ctp'].append(100*ct/total)
        error5w['cgapp'].append(100*cgap/total)
        error5w['ccp'].append(100*cc/total)
        error5w['gap'].append(100*ga/total)
        error5w['gcp'].append(100*gc/total)
        error5w['gtp'].append(100*gt/total)
        error5w['ggapp'].append(100*ggap/total)
        error5w['ggp'].append(100*gg/total)
        error5w['tap'].append(100*ta/total)
        error5w['tcp'].append(100*tc/total)
        error5w['tgp'].append(100*tg/total)
        error5w['tgapp'].append(100*tgap/total)
        error5w['ttp'].append(100*tt/total)
        error5w['gapap'].append(100*gapa/total)
        error5w['gapcp'].append(100*gapc/total)
        error5w['gapgp'].append(100*gapg/total)
        error5w['gaptp'].append(100*gapt/total)
        error5w['alnslengths'].append(total)
        error5w['expeca'].append(100*tmpa/total)
        error5w['expecc'].append(100*tmpc/total)
        error5w['expecg'].append(100*tmpg/total)
        error5w['expect'].append(100*tmpt/total)
    error5w_frame = pd.DataFrame.from_dict(error5w, orient='index', columns=error5w['keys'])
    error5w_frame = error5w_frame.transpose()
    error5w_frame.to_csv(mothur_query_fasta[:-5]+reference_bed_output[-19:-15]+'dataframe.csv')

In [ ]:
# pairwise_aln('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.reference.ITS.bedoutput.fasta', '/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.filtered_base_2000-4000.fasta.mothur5.ITS.pcr.fasta')

In [ ]:
def plot_stats_ind(filename):
    
    frame = pd.DataFrame.from_csv(filename)
    result5w = {}
    result5w_key_entries = ['astats', 'cstats', 'gstats', 'tstats', 'gapstats', 'expec', 'ostats']
    for item in result5w_key_entries:
        result5w[item] = []
    result5w['astats'] = [frame['aap'], frame['acp'], frame['agp'], frame['atp'], frame['agapp']]
    result5w['cstats'] = [frame['cap'], frame['ccp'], frame['cgp'], frame['ctp'], frame['cgapp']]
    result5w['gstats'] = [frame['gap'], frame['gcp'], frame['ggp'], frame['gtp'], frame['ggapp']]
    result5w['tstats'] = [frame['tap'], frame['tcp'], frame['tgp'], frame['ttp'], frame['tgapp']]
    result5w['gapstats'] = [frame['gapap'], frame['gapcp'], frame['gapgp'], frame['gaptp']]
    
    fg, ax = plt.subplots(5, 5, figsize=(20,20))
    fg.suptitle("Base Percentages", fontsize=28, y=1.35)
    fg.subplots_adjust(top=1.3)
    titles = ['A-A', 'A-C', 'A-G', 'A-T', 'A-GAP', 'C-A', 'C-C', 'C-G', 'C-T', 'C-GAP', 'G-A', 'G-C', 'G-G', 'G-T', 'G-GAP', 'T-A', 'T-C', 'T-G', 'T-T', 'T-GAP', 'GAP-A', 'GAP-C', 'GAP-G', 'GAP-T']

    for i in range(0, 24):
        if i < 5:
            sns.distplot(result5w['astats'][i], color='k', ax=ax[0][i])
            ax[0][i].set_title(titles[i], fontsize=17)
            ax[0][i].set(xlabel='percentage of changes', ylabel='frequency')
        elif i > 4 and i < 10:
            sns.distplot(result5w['cstats'][i-5], color='k', ax=ax[1][i-5])
            ax[1][i-5].set_title(titles[i], fontsize=17)
            ax[1][i-5].set(xlabel='percentage of changes', ylabel='frequency')
        elif i > 9 and i < 15:
            sns.distplot(result5w['gstats'][i-10], color='k', ax=ax[2][i-10])
            ax[2][i-10].set_title(titles[i], fontsize=17)
            ax[2][i-10].set(xlabel='percentage of changes', ylabel='frequency')
        elif i > 14 and i < 20:
            sns.distplot(result5w['tstats'][i-15], color='k', ax=ax[3][i-15])
            ax[3][i-15].set_title(titles[i], fontsize=17)
            ax[3][i-15].set(xlabel='percentage of changes', ylabel='frequency')
        else:
            sns.distplot(result5w['gapstats'][i-20], color='k', ax=ax[4][i-20])
            ax[4][i-20].set_title(titles[i], fontsize=17)
            ax[4][i-20].set(xlabel='percentage of changes', ylabel='frequency')
    fg.savefig(filename[:-3]+'ind_stats.png', bbox_inches='tight')
    return filename[:-3]+'ind_stats.png'

In [ ]:
# plot_stats_ind('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.filtered_base_2000-4000.fasta.mothur5.ITS.pcr.ITS.dataframe.csv')

In [ ]:
def plot_stats_gen(filename):
    frame = pd.DataFrame.from_csv(filename)
    result5w = {}
    result5w_key_entries = ['expec', 'ostats', 'omatches']
    for item in result5w_key_entries:
        result5w[item] = []
    result5w['expec'] = [frame['expeca'], frame['expecc'], frame['expecg'], frame['expect']]
    for i in range(0, len(frame['aap'])):
        result5w['omatches'].append(frame['aap'][i]+frame['ccp'][i]+frame['ggp'][i]+frame['ttp'][i])
    result5w['ostats'] = [result5w['omatches'], frame['aap'], frame['ccp'], frame['ggp'], frame['ttp']]
    
    fg, ax = plt.subplots(5, 1, figsize=(20,20), squeeze=False)
    fg.suptitle("Base Percentages", fontsize=28, y=1.35)
    fg.subplots_adjust(top=1.3)
    otitles=['Overall Matches', 'A Matches', 'C Matches', 'G Matches', 'T Matches']

    for i in range(0, len(result5w['ostats'])):
        sns.distplot(result5w['ostats'][i], color='k', ax=ax[i][0])
        ax[i][0].set_title(otitles[i], fontsize=17)
        ax[i][0].set(xlabel='percentage of changes', ylabel='frequency')
        if i > 0:
            ax[i][0].set(xlim=(14, 30))
            ax[i][0].axvline(np.mean(result5w['expec'][i-1]))
    fg.savefig(filename[:-3]+'gen_stats.png', bbox_inches='tight')
    return filename[:-3]+'gen_stats.png'

In [ ]:
# plot_stats_gen('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.filtered_base_2000-4000.fasta.mothur5.ITS.pcr.ITS.dataframe.csv')

In [ ]:
def run_ITS(fastq, reference_fasta, primer_fasta):
    get_fasta(fastq)
    print('get_fasta DONE')
    get_bedoutput_ITS(reference_fasta, primer_fasta)
    print('get_bedoutput_ITS DONE')
    get_bedfasta(reference_fasta[:-6]+'.ITS.bedfile', reference_fasta)
    print('get_bedfasta DONE')
    length_filter(fastq[:-6]+'.fasta', 2000, 4000)
    print('length_filter DONE')
    pcr_seqs((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta', primer_fasta[:-14]+'.oligos')
    print('pcr_seqs DONE')
    pairwise_aln(reference_fasta[:-6]+'.ITS.bedoutput.fasta', ((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta')[:-6]+'.mothur5.%s.pcr.fasta' % ((primer_fasta[:-14]+'.oligos')[-10:-7]))
    plot_stats_ind(((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta')+'.mothur5.%s.pcr.fasta'[:-5] % ((primer_fasta[:-14]+'.oligos')[-10:-7])+(reference_fasta[:-6]+'.ITS.bedoutput.fasta')[-19:-15]+'dataframe.csv')
    plot_stats_gen(((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta')+'.mothur5.%s.pcr.fasta'[:-5] % ((primer_fasta[:-14]+'.oligos')[-10:-7])+(reference_fasta[:-6]+'.ITS.bedoutput.fasta')[-19:-15]+'dataframe.csv')

In [ ]:
run_ITS('/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.fastq', '/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/zymoseptoria_tritici.reference.fasta', '/media/MassStorage/tmp/TE/summer_project/general_analysis/zymoseptoria_tritici/ITS_primers.fasta')

In [11]:
%%writefile run_ITS.py

import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import seaborn as sns
import matplotlib.pyplot as plt
import subprocess
import os
from mothur_py import Mothur
from shutil import copy
import warnings
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from ipysankeywidget import SankeyWidget
warnings.filterwarnings("ignore")

def get_fasta(fastq):
    SeqIO.convert(fastq, "fastq", fastq[:-6]+'.fasta', "fasta")
    return fastq[:-6]+'.fasta'

def get_bedoutput_ITS(reference_fasta, primer_fasta):
    """The purpose of this function is to generate a fasta file of the reference genome regions between primers, 
    which should correspond with the region sequenced using the MinION. This requires first the generation of a 
    database using the nucleotide makeblastdb command, followed by running a blast search across the database 
    using the primer sequence. These command line commands are processed through subprocess. Subsequently, the 
    resulting outfile from the blast is read in as a dataframe and this dataframe is manipulated in order to 
    utilise only the minimum evalues. Each forward sequence is then compared to each reverse sequence, given they
    are on the same contig and the resultant read would be within 2000-4000 bp long. These results are saved as 
    a bedfile for use in the get_bedfasta function"""
    cmd = 'makeblastdb -in %s -dbtype nucl -out %s' % (reference_fasta, reference_fasta[:-6]+'_db')
    subprocess.getoutput(cmd)
    cmd2 = 'blastn -query %s -db %s -evalue=100000 -task "blastn-short" -outfmt 6 > %s' % (primer_fasta, reference_fasta[:-6]+'_db', reference_fasta[:-6]+'.outfmt6')
    subprocess.getoutput(cmd2)
    df = pd.read_csv(reference_fasta[:-6]+'.outfmt6', sep="\t", header=None, names=["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"])
    forward_df = df.loc[(df['qseqid']=='forward')]
    forward_df = forward_df.loc[df['evalue']==min(forward_df['evalue'])]
    forward_df = forward_df.reset_index()
    reverse_df = df.loc[(df['qseqid']=='reverse')]
    reverse_df = reverse_df.loc[df['evalue']==min(reverse_df['evalue'])]
    reverse_df = reverse_df.reset_index()
    forward_bed = pd.DataFrame(columns=['chrom', 'chromStart', 'chromEnd'])
    for i in range(0, len(forward_df)):
        if forward_df['sstart'][i] <= forward_df['send'][i]:
            forward_bed = forward_bed.append({'chrom': forward_df['sseqid'][i],'chromStart': (forward_df['sstart'][i])-1, 'chromEnd': forward_df['send'][i]}, ignore_index=True)
        else:
            forward_bed = forward_bed.append({'chrom': forward_df['sseqid'][i],'chromStart': (forward_df['send'][i])-1, 'chromEnd': forward_df['sstart'][i]}, ignore_index=True)
    forward_bed.sort_values(['chromStart'])
    reverse_bed = pd.DataFrame(columns=['chrom', 'chromStart', 'chromEnd'])
    for i in reverse_df.index:
        if reverse_df['sstart'][i] < reverse_df['send'][i]:
            reverse_bed = reverse_bed.append({'chrom': reverse_df['sseqid'][i],'chromStart': (reverse_df['sstart'][i])-1, 'chromEnd': reverse_df['send'][i]}, ignore_index=True)
        else:
            reverse_bed = reverse_bed.append({'chrom': reverse_df['sseqid'][i],'chromStart': (reverse_df['send'][i])-1, 'chromEnd': reverse_df['sstart'][i]}, ignore_index=True)
    reverse_bed.sort_values(['chromStart'])
    intervals_list = []
    for row in forward_bed.itertuples(index=True, name='Pandas'):
        counter = 0
        for rows in reverse_bed.itertuples(index=True, name='Pandas'):
            # Here lies the difference between the ITS and TEF versions of this function - for ITS the read length range is 2000-4000
            if getattr(row, 'chrom') == getattr(rows, 'chrom') and 2000 < np.absolute(getattr(row, 'chromEnd') - getattr(rows, 'chromStart')) < 4000 and getattr(row, 'chromEnd') > getattr(rows, 'chromStart'):
                counter += 1
                intervals_list.append([forward_bed['chrom'][getattr(row, 'Index')], reverse_bed['chromStart'][getattr(rows, 'Index')], forward_bed['chromEnd'][getattr(row, 'Index')]])
            elif getattr(row, 'chrom') == getattr(rows, 'chrom') and 2000 < np.absolute(getattr(row, 'chromEnd') - getattr(rows, 'chromStart')) < 4000 and getattr(row, 'chromEnd') < getattr(rows, 'chromStart'):
                intervals_list.append([forward_bed['chrom'][getattr(row, 'Index')], forward_bed['chromEnd'][getattr(row, 'Index')], reverse_bed['chromStart'][getattr(rows, 'Index')]])
        if counter > 1:
            print("WARNING")
    intervals_frame = pd.DataFrame(data=intervals_list, columns=['chrom', 'chromStart', 'chromEnd'])
    print(intervals_frame)
    intervals_frame.to_csv(reference_fasta[:-6]+'.ITS.bedfile', sep='\t', header=False, index=False)
#     lines = []
#     for i in range(0, len(intervals_frame)):
#         lines.append(['%i\t%i\t%i' % (intervals_frame['chrom'][i], intervals_frame['chromStart'][i], intervals_frame['chromEnd'][i])]) 
#     with open(reference_fasta[:-6]+'.ITS.bedfile', 'w') as f:
#         for i in range(0, len(lines)):
#             f.writelines(lines[i])
#             f.writelines('\n')
    return reference_fasta[:-6]+'.ITS.bedfile'

def get_bedfasta(bedfile, reference_fasta):
    cmd2 = 'bedtools getfasta -fo %s -fi %s -bed %s' % (reference_fasta[:-6]+'.ITS.bedoutput.fasta', reference_fasta, bedfile)
    subprocess.getoutput(cmd2)
    return reference_fasta[:-6]+'.ITS.bedoutput.fasta'

def length_filter(fasta, minimum, maximum):
    fasta_dict = SeqIO.to_dict(SeqIO.parse(fasta, "fasta"))
    short_dict = fasta_dict.copy()
    for key in fasta_dict:
        if len(fasta_dict[key].seq) not in range(minimum, maximum + 1):
            del short_dict[key]
    SeqIO.write(short_dict.values(), fasta[:-6]+'.filtered_base_%s-%s.fasta' % (minimum, maximum), "fasta")
    return fasta[:-6]+'.filtered_base_%s-%s.fasta' % (minimum, maximum)

def pcr_seqs(filtered_fasta, oligos):
    tmp = SeqIO.to_dict(SeqIO.parse(filtered_fasta, "fasta"))
    new_name = filtered_fasta+'.mothur5.%s.fasta' % (oligos[-10:-7])
    SeqIO.write(tmp.values(), new_name, "fasta")
    m = Mothur()
    m.pcr.seqs(fasta=new_name, oligos=oligos, pdiffs=5, rdiffs=5)
    return new_name
    
def pairwise_aln(reference_bed_output, mothur_query_fasta):
    gen_dict = SeqIO.to_dict(SeqIO.parse(reference_bed_output, "fasta"))
    print(gen_dict)
    gen_dict_seqs = []
    for key in gen_dict:
        gen_dict_seqs.append(gen_dict[key].seq)
    print(gen_dict_seqs)
    q_dict = SeqIO.to_dict(SeqIO.parse(mothur_query_fasta, "fasta"))
    error5w = {}
    error5w_key_entries = ['keys', 'alnslengths', 'aap', 'acp', 'agp', 'atp', 'agapp', 'cap', 'ccp', 'cgp', 'ctp', 'cgapp', 'gap', 'gcp', 'ggp', 'gtp', 'ggapp', 'tap', 'tcp', 'tgp', 'ttp', 'tgapp', 'gapap', 'gapcp', 'gapgp', 'gaptp', 'expeca', 'expecc', 'expecg', 'expect']
    for item in error5w_key_entries:
        error5w[item] = []
    for key in q_dict:
        error5w['keys'].append(key)
        alignments = pairwise2.align.globalms(gen_dict_seqs[0], q_dict[key].seq, 1, -1, -1, 0)
        total = 0
        aa = 0
        ac = 0
        ag = 0
        at = 0
        agap = 0
        ca =0
        cc=0
        cg=0
        ct=0
        cgap=0
        ga=0
        gc=0
        gg=0
        gt=0
        ggap=0
        ta=0
        tc=0
        tg=0
        tt=0
        tgap=0
        gapa=0
        gapc=0
        gapg=0
        gapt=0
        tmpa=0
        tmpc=0
        tmpg=0
        tmpt=0
        comparison = alignments[0][0:2]
        for i in range(0, len(comparison[0])):
            if comparison[0][i] == "A":
                tmpa += 1
                if comparison[1][i] == "C":
                    ac+=1
                    total += 1
                elif comparison[1][i] == "G":
                    ag+=1
                    total += 1
                elif comparison[1][i] == "T":
                    at+=1
                    total += 1
                elif comparison[1][i] == "A":
                    aa+=1
                    total += 1
                else:
                    agap+=1
                    total += 1
            if comparison[0][i] == "C":
                tmpc+=1
                if comparison[1][i] == "A":
                    ca+=1
                    total += 1
                elif comparison[1][i] == "G":
                    cg+=1
                    total += 1
                elif comparison[1][i] == "T":
                    ct+=1
                    total += 1
                elif comparison[1][i] == "C":
                    cc+=1
                    total += 1
                else:
                    cgap+=1
                    total += 1
            if comparison[0][i] == "G":
                tmpg+=1
                if comparison[1][i] == "A":
                    ga+=1
                    total += 1
                elif comparison[1][i] == "C":
                    gc+=1
                    total += 1
                elif comparison[1][i] == "T":
                    gt+=1
                    total += 1
                elif comparison[1][i] == "G":
                    gg+=1
                    total += 1
                else:
                    ggap+=1
                    total += 1
            if comparison[0][i] == "T":
                tmpt+=1
                if comparison[1][i] == "A":
                    ta+=1
                    total += 1
                elif comparison[1][i] == "C":
                    tc+=1
                    total += 1
                elif comparison[1][i] == "G":
                    tg+=1
                    total += 1
                elif comparison[1][i] == "T":
                    tt+=1
                    total += 1
                else:
                    tgap+=1
                    total += 1
            if comparison[0][i] == "-":
                if comparison[1][i] == "A":
                    gapa+=1
                    total += 1
                elif comparison[1][i] == "C":
                    gapc+=1
                    total += 1
                elif comparison[1][i] == "G":
                    gapg+=1
                    total += 1
                elif comparison[1][i] == "T":
                    gapt+=1
                    total += 1
        error5w['acp'].append(100*ac/total)
        error5w['agp'].append(100*ag/total)
        error5w['atp'].append(100*at/total)
        error5w['agapp'].append(100*agap/total)
        error5w['aap'].append(100*aa/total)
        error5w['cap'].append(100*ca/total)
        error5w['cgp'].append(100*cg/total)
        error5w['ctp'].append(100*ct/total)
        error5w['cgapp'].append(100*cgap/total)
        error5w['ccp'].append(100*cc/total)
        error5w['gap'].append(100*ga/total)
        error5w['gcp'].append(100*gc/total)
        error5w['gtp'].append(100*gt/total)
        error5w['ggapp'].append(100*ggap/total)
        error5w['ggp'].append(100*gg/total)
        error5w['tap'].append(100*ta/total)
        error5w['tcp'].append(100*tc/total)
        error5w['tgp'].append(100*tg/total)
        error5w['tgapp'].append(100*tgap/total)
        error5w['ttp'].append(100*tt/total)
        error5w['gapap'].append(100*gapa/total)
        error5w['gapcp'].append(100*gapc/total)
        error5w['gapgp'].append(100*gapg/total)
        error5w['gaptp'].append(100*gapt/total)
        error5w['alnslengths'].append(total)
        error5w['expeca'].append(100*tmpa/total)
        error5w['expecc'].append(100*tmpc/total)
        error5w['expecg'].append(100*tmpg/total)
        error5w['expect'].append(100*tmpt/total)
    error5w_frame = pd.DataFrame.from_dict(error5w, orient='index', columns=error5w['keys'])
    error5w_frame = error5w_frame.transpose()
    error5w_frame.to_csv(mothur_query_fasta[:-5]+reference_bed_output[-19:-15]+'dataframe.csv')
    return mothur_query_fasta[:-5]+reference_bed_output[-19:-15]+'dataframe.csv'

def plot_stats_ind(filename):
    
    frame = pd.DataFrame.from_csv(filename)
    result5w = {}
    result5w_key_entries = ['astats', 'cstats', 'gstats', 'tstats', 'gapstats', 'expec', 'ostats']
    for item in result5w_key_entries:
        result5w[item] = []
    result5w['astats'] = [frame['aap'], frame['acp'], frame['agp'], frame['atp'], frame['agapp']]
    result5w['cstats'] = [frame['cap'], frame['ccp'], frame['cgp'], frame['ctp'], frame['cgapp']]
    result5w['gstats'] = [frame['gap'], frame['gcp'], frame['ggp'], frame['gtp'], frame['ggapp']]
    result5w['tstats'] = [frame['tap'], frame['tcp'], frame['tgp'], frame['ttp'], frame['tgapp']]
    result5w['gapstats'] = [frame['gapap'], frame['gapcp'], frame['gapgp'], frame['gaptp']]
    
    fg, ax = plt.subplots(5, 5, figsize=(20,20))
    fg.suptitle("Base Percentages", fontsize=28, y=1.35)
    fg.subplots_adjust(top=1.3)
    titles = ['A-A', 'A-C', 'A-G', 'A-T', 'A-GAP', 'C-A', 'C-C', 'C-G', 'C-T', 'C-GAP', 'G-A', 'G-C', 'G-G', 'G-T', 'G-GAP', 'T-A', 'T-C', 'T-G', 'T-T', 'T-GAP', 'GAP-A', 'GAP-C', 'GAP-G', 'GAP-T']

    for i in range(0, 24):
        if i < 5:
            sns.distplot(result5w['astats'][i], color='k', ax=ax[0][i])
            ax[0][i].set_title(titles[i], fontsize=17)
            ax[0][i].set(xlabel='percentage of changes', ylabel='frequency')
        elif i > 4 and i < 10:
            sns.distplot(result5w['cstats'][i-5], color='k', ax=ax[1][i-5])
            ax[1][i-5].set_title(titles[i], fontsize=17)
            ax[1][i-5].set(xlabel='percentage of changes', ylabel='frequency')
        elif i > 9 and i < 15:
            sns.distplot(result5w['gstats'][i-10], color='k', ax=ax[2][i-10])
            ax[2][i-10].set_title(titles[i], fontsize=17)
            ax[2][i-10].set(xlabel='percentage of changes', ylabel='frequency')
        elif i > 14 and i < 20:
            sns.distplot(result5w['tstats'][i-15], color='k', ax=ax[3][i-15])
            ax[3][i-15].set_title(titles[i], fontsize=17)
            ax[3][i-15].set(xlabel='percentage of changes', ylabel='frequency')
        else:
            sns.distplot(result5w['gapstats'][i-20], color='k', ax=ax[4][i-20])
            ax[4][i-20].set_title(titles[i], fontsize=17)
            ax[4][i-20].set(xlabel='percentage of changes', ylabel='frequency')
    fg.savefig(filename[:-3]+'ind_stats.png', bbox_inches='tight')
    return filename[:-3]+'ind_stats.png'

def plot_stats_gen(filename):
    frame = pd.DataFrame.from_csv(filename)
    result5w = {}
    result5w_key_entries = ['expec', 'ostats', 'omatches']
    for item in result5w_key_entries:
        result5w[item] = []
    result5w['expec'] = [frame['expeca'], frame['expecc'], frame['expecg'], frame['expect']]
    for i in range(0, len(frame['aap'])):
        result5w['omatches'].append(frame['aap'][i]+frame['ccp'][i]+frame['ggp'][i]+frame['ttp'][i])
    result5w['ostats'] = [result5w['omatches'], frame['aap'], frame['ccp'], frame['ggp'], frame['ttp']]
    
    fg, ax = plt.subplots(5, 1, figsize=(20,20), squeeze=False)
    fg.suptitle("Base Percentages", fontsize=28, y=1.35)
    fg.subplots_adjust(top=1.3)
    otitles=['Overall Matches', 'A Matches', 'C Matches', 'G Matches', 'T Matches']

    for i in range(0, len(result5w['ostats'])):
        sns.distplot(result5w['ostats'][i], color='k', ax=ax[i][0])
        ax[i][0].set_title(otitles[i], fontsize=17)
        ax[i][0].set(xlabel='percentage of changes', ylabel='frequency')
        if i > 0:
            ax[i][0].set(xlim=(14, 30))
            ax[i][0].axvline(np.mean(result5w['expec'][i-1]))
    fg.savefig(filename[:-3]+'gen_stats.png', bbox_inches='tight')
    
    ostats = [[np.mean(result5w['ostats'][0]), np.median(result5w['ostats'][0])], [np.mean(result5w['ostats'][1]), np.median(result5w['ostats'][1])], [np.mean(result5w['ostats'][2]), np.median(result5w['ostats'][2])], [np.mean(result5w['ostats'][3]), np.median(result5w['ostats'][3])], [np.mean(result5w['ostats'][4]), np.median(result5w['ostats'][4])]]
    ostatsframe = pd.DataFrame(data=ostats, columns=['Overall Matches', 'A Matches', 'C Matches', 'G Matches', 'T Matches'], index=['Mean', 'Median'])
    ostatsframe.to_csv(filename[:-3]+'base_stats.csv')
    return filename[:-3]+'gen_stats.png'

def run_ITS(fastq, reference_fasta, primer_fasta):
    get_fasta(fastq)
    print('get_fasta DONE')
    get_bedoutput_ITS(reference_fasta, primer_fasta)
    print('get_bedoutput_ITS DONE')
    get_bedfasta(reference_fasta[:-6]+'.ITS.bedfile', reference_fasta)
    print('get_bedfasta DONE')
    length_filter(fastq[:-6]+'.fasta', 2000, 4000)
    print('length_filter DONE')
    pcr_seqs((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta', primer_fasta[:-14]+'.oligos')
    print('pcr_seqs DONE')
    pairwise_aln(reference_fasta[:-6]+'.ITS.bedoutput.fasta', ((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta')+'.mothur5.%s.pcr.fasta' % ((primer_fasta[:-14]+'.oligos')[-10:-7]))
    print('pairwise alignments DONE')
    plot_stats_ind(((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta')+'.mothur5.%s.pcr.fasta'[:-5] % ((primer_fasta[:-14]+'.oligos')[-10:-7])+(reference_fasta[:-6]+'.ITS.bedoutput.fasta')[-19:-15]+'dataframe.csv')
    plot_stats_gen(((fastq[:-6]+'.fasta')[:-6]+'.filtered_base_2000-4000.fasta')+'.mothur5.%s.pcr.fasta'[:-5] % ((primer_fasta[:-14]+'.oligos')[-10:-7])+(reference_fasta[:-6]+'.ITS.bedoutput.fasta')[-19:-15]+'dataframe.csv')
    

Overwriting run_ITS.py
